In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
from sklearn.datasets import load_boston

In [22]:
boston = load_boston()
X = boston['data']
y = boston['target']
trainSize = len(X)/4*3
X_train = X[:trainSize]
y_train = y[:trainSize]
X_test = X[trainSize:]
y_test = y[trainSize:]

In [51]:
from numpy import std

In [302]:
def partition(Q, q, j, t):
    L_indexes = [i for i in range(0, len(q)) if Q[i, j] <= t]
    L_X = np.array([Q[i] for i in L_indexes])
    L_y = np.array([q[i] for i in L_indexes])
    R_indexes = [i for i in range(0, len(q)) if Q[i, j] > t]
    R_X = np.array([Q[i] for i in R_indexes])
    R_y = np.array([q[i] for i in R_indexes])
    return L_X, L_y, R_X, R_y

def G(Q, q, param):
    j, t = param
    L_X, L_y, R_X, R_y = partition(Q, q, j, t)
    return float(len(L_y))/len(q)*std(L_y) + float(len(R_y))/len(q)*std(R_y)

In [340]:
class Node:
    def __init__(self, j=-1, t=0):
        self.j = j
        self.t = t
        self.left = 0
        self.right = 0
        self.parent = 0
        self.depth = 0
        self.answer = 0
        self.leaf = 0
        
    def set_right(self, j=-1, t=0):
        self.right = Node(j, t)
        self.right.parent = self
        
    def set_left(self, j=-1, t=0):
        self.left = Node(j, t)
        self.left.parent = self
        
    def set_node(self, X, y):
        jt = []
        for j in range(0, 13):
            feature = np.sort(list(set((X[:,j]))))
            for t in [(feature[i] + feature[i-1]) / 2. for i in range(1, len(feature))]:
                jt.append((j, t))
        G_values = [G(X, y, jt[i]) for i in range(0, len(jt))]
        j, t = jt[np.argmin(G_values)]
        self.j = j
        self.t = t
        if self.parent != 0:
            self.depth = self.parent.depth + 1
        self.set_right()
        self.set_left()
        return partition(X, y, j, t)
    
    def get_depth(self):
        return self.depth
    
    def set_leaf(self, y):
        self.leaf = 1
        self.answer = np.average(y)

In [366]:
class DesigionTree:
    def __init__(self, max_depth):
        self.max_depth = max_depth
        self.root = Node()
    
    def build(self, current_node, X, y):            
        L_X, L_y, R_X, R_y = current_node.set_node(X, y)
        if current_node.get_depth() == self.max_depth:
            current_node.right.set_leaf(R_y)
            current_node.left.set_leaf(L_y)
            return 
        if len(R_X) <= 1:
            current_node.right.set_leaf(R_y)
        else:
            self.build(current_node.right, R_X, R_y)
        if len(L_X) <= 1:
            current_node.left.set_leaf(L_y)
        else:
            self.build(current_node.left, L_X, L_y)
        
    def fit(self, X, y):
        current_node = self.root
        self.build(current_node, X, y)
        
    def predict(self, X):
        y = np.zeros(len(X))
        for i in range(0, len(X)):
            x = X[i]
            current = self.root
            while current.leaf == 0:
                if x[current.j] <= current.t:
                    current = current.left
                else:
                    current = current.right
            y[i] = current.answer
        return y

In [375]:
tree = DesigionTree(10)
tree.fit(X_train,y_train)
y0 = tree.predict(X_train)
mean_squared_error(y0, y_train)

0.38550072082705128